In [14]:
from optimum.intel.openvino import OVStableDiffusionXLPipeline
# from nncf import compress_weights, CompressWeightsMode
model_id = "stabilityai/stable-diffusion-xl-base-1.0"


ov_pipe_bf16 = OVStableDiffusionXLPipeline.from_pretrained(model_id, compile=False, export=True, load_in_8bit=False)
# ov_pipe_bf16
type(ov_pipe_bf16.text_encoder.model)


Framework not specified. Using pt to export to ONNX.
Keyword arguments {'subfolder': '', 'trust_remote_code': False} are not expected by StableDiffusionXLImg2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Using framework PyTorch: 2.1.0+cu121
/home/devuser/nlyalyus/env/lm-eval-repro/lib/python3.8/site-packages/transformers/modeling_attn_mask_utils.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/home/devuser/nlyalyus/env/lm-eval-repro/lib/python3.8/site-packages/transformers/modeling_attn_mask_utils.py:161: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
/home/devuser/nlyalyus/env/lm-eval-repro/lib/python3.8/site-packages/transformers/models/clip/modeling_clip.py

openvino.runtime.ie_api.Model

In [3]:
print(ov_pipe_bf16)

OVStableDiffusionXLPipeline {
  "_class_name": "OVStableDiffusionXLPipeline",
  "_diffusers_version": "0.21.4",
  "add_watermarker": null,
  "force_zeros_for_empty_prompt": true,
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "optimum",
    "OVModelTextEncoder"
  ],
  "text_encoder_2": [
    "optimum",
    "OVModelTextEncoder"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "optimum",
    "OVModelUnet"
  ],
  "vae_decoder": [
    "optimum",
    "OVModelVaeDecoder"
  ],
  "vae_encoder": [
    "optimum",
    "OVModelVaeEncoder"
  ]
}



In [10]:
from optimum.intel import OVQuantizer
from optimum.intel import OVConfig
quantizer = OVQuantizer.from_pretrained(ov_pipe_bf16.text_encoder.model)
quantizer.quantize(save_directory='', weights_only=True, quantization_config=OVConfig(compression={"type": "int4_sym_g128", "ratio": 0.8}),)

AttributeError: 'Model' object has no attribute 'forward'

In [13]:
type(ov_pipe_bf16.text_encoder.model)

AttributeError: 'Model' object has no attribute 'text_encoder'

In [12]:
from nncf import compress_weights, CompressWeightsMode
compressed_text_encoder = compress_weights(ov_pipe_bf16.text_encoder.model, mode=CompressWeightsMode.INT4_SYM, group_size=128, ratio=0.8)

Output()

INFO:nncf:Statistics of the bitwidth distribution:
+--------------+---------------------------+-----------------------------------+
| Num bits (N) | % all parameters (layers) |    % ratio-defining parameters    |
|              |                           |             (layers)              |
+==============+===========================+===================================+
| 8            | 47% (30 / 74)             | 21% (27 / 71)                     |
+--------------+---------------------------+-----------------------------------+
| 4            | 53% (44 / 74)             | 79% (44 / 71)                     |
+--------------+---------------------------+-----------------------------------+


Output()

In [37]:
from pathlib import Path
path = Path('/dev/data/nlyalyus/cache/opt-6.7b/int4_asym_g64')
from openvino import Core
core = Core()

In [38]:
compressed_model = core.read_model(model=str(path / 'openvino_model.xml'))

In [43]:
actual_stats = {}
import numpy as np
for op in compressed_model.get_ops():
    op_name = op.get_friendly_name()
    if op.get_type_name() == "Constant" and op.get_element_type() == ov.Type(np.uint8) and 'ZP' not in op_name:
        weight = op.data
        print(op_name, op.get_shape(), weight[:10], (weight.size - np.count_nonzero(weight)) / weight.size * 100)

Constant_571577_compressed [4096, 64, 64] [[[11  7  6 ... 11  6  9]
  [ 5  6 13 ...  8 12 11]
  [ 9 10 10 ...  8 11  5]
  ...
  [ 8  6  7 ... 14  8 10]
  [ 3  0 11 ...  8 11 10]
  [ 9  8  9 ...  5  8  7]]

 [[ 7  9  4 ... 13  4  5]
  [ 7 10  7 ...  9  4  9]
  [11 12  8 ... 11 12  7]
  ...
  [ 6  7  9 ...  5  7  5]
  [12  5  7 ...  0 13 10]
  [ 5  3 13 ...  7  9  9]]

 [[ 6  9  8 ...  9  9  9]
  [ 5  6 12 ...  7 10  0]
  [12  9  5 ...  9 12  9]
  ...
  [10 13  0 ...  6 10  9]
  [ 2  1  3 ... 14  8  8]
  [13 14  8 ...  6 11  9]]

 ...

 [[ 9  0  6 ...  9 12  9]
  [10 11  9 ... 10  4 11]
  [ 8  8 10 ...  6  6  9]
  ...
  [ 4  4 14 ...  7  4  5]
  [12 10 10 ... 12  6  9]
  [ 3 10  6 ...  8  8 10]]

 [[ 9  6  5 ... 10  6  4]
  [ 9  0  8 ... 14  3  6]
  [ 7  5  8 ... 15  6  8]
  ...
  [ 8 11 13 ...  5  3 12]
  [ 2 15  8 ...  5  1  5]
  [10  5  8 ... 13  6  6]]

 [[ 1 15 10 ...  5 10 13]
  [ 9 12  0 ...  5  3  2]
  [ 7  9 13 ... 14  9 12]
  ...
  [ 5  6 11 ...  3  6  6]
  [ 9 10  9 ...  3 10 

In [81]:
from datasets import load_dataset
# dataset = load_dataset("liwu/MNBVC", 'law_judgement', split='train', streaming=True)
# dataset = load_dataset("indiejoseph/wikitext-zh-yue", split='test', streaming=True)
dataset = load_dataset("allenai/c4", subset='zh', split='validation', streaming=True, data_file='multilingual/c4-zh.tfrecord-00000-of-01024.json.gz')


next(iter(dataset))

KeyboardInterrupt: 

In [50]:
print(dataset)
iter_data = iter(dataset)
next(iter_data)
next(iter_data)


{'text': '紐約兩大機場之一嘅拉瓜迪亞機場係以佢來命名嘅。'}

In [75]:
next(iter_data)

{'text': '好多認知心理學家都有就「突然出現一個諗頭打前，個腦到底發生咗咩事」作出假說：'}

In [93]:
name='Qwen/Qwen-7B-Chat'
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/home/devuser/nlyalyus/projects/lm-evaluation-harness/cache/qwen-7b-chat/gptq', trust_remote_code=True, allowed_special='all')

In [105]:
tokenizer.special_tokens
tokenizer.unk_token_id
tokenizer.eos_token_id
tokenizer.eod_id

151643

In [113]:
s = '追书神器，小说追新更神器，汇集全网实时热门小说，5分钟内快速更新，上亿书友共享，拒绝书荒全网小说光速追更提醒，全站阅读无广告，多维推荐送好书，更有集吐槽书评交友等多位一体的高能社区，超好用的网文小说电子书追更神器更新内容V4.37新增新用户福利，新设备最多可15天免费读新增全新页面及功能调整，提高视觉与操作体验优化优化VIP专区的阅读体验修复修复上个版本已知问题'
l = tokenizer.batch_decode(tokenizer(s).input_ids)[-1]
len(l)

2